<a href="https://colab.research.google.com/github/Tiberius42/wheat-heads/blob/master/Wheat_MaskRCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sat Jul  4 01:58:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#self.keras_model.add_metric(loss, name)
!git clone https://github.com/matterport/Mask_RCNN.git

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 116.76 MiB | 39.58 MiB/s, done.
Resolving deltas: 100% (566/566), done.


In [ ]:
%tensorflow_version 1.x

#!pip install tensorflow==1.15.0
!pip install q keras==2.1.0

TensorFlow 1.x selected.
     |████████████████████████████████| 307kB 12.2MB/s 
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.1.0 which is incompatible.
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
!gdown --id 1j2Ki-sMTsIrJHi15Yb-eDJXylGpy_ob1

Downloading...
From: https://drive.google.com/uc?id=1j2Ki-sMTsIrJHi15Yb-eDJXylGpy_ob1
To: /content/train.csv
8.32MB [00:00, 50.8MB/s]


In [ ]:
!gdown --id 1xHaWSvS-Zqt5MkGXlG_KoiLMHrpFMDBX

Downloading...
From: https://drive.google.com/uc?id=1xHaWSvS-Zqt5MkGXlG_KoiLMHrpFMDBX
To: /content/mask_rcnn_coco.h5
258MB [00:04, 56.9MB/s]


In [ ]:
import pandas as pd
import numpy as np
# from tensorflow.keras import backend
%cd Mask_RCNN
from mrcnn.utils import Dataset, extract_bboxes
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.visualize import display_instances
%cd ..
import os
import matplotlib

import keras
print(keras.__version__)

/content/Mask_RCNN
/content
2.1.0


Using TensorFlow backend.


In [ ]:
%cd Mask_RCNN
!pwd
!python setup.py install
%cd ..

/content/Mask_RCNN
/content/Mask_RCNN
running install
running bdist_egg
running egg_info
creating mask_rcnn.egg-info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/mrcnn
copying mrcnn/config.py -> build/lib/mrcnn
copying mrcnn/__init__.py -> build/lib/mrcnn
copying mrcnn/model.py -> build/lib/mrcnn
copying mrcnn/utils.py -> build/lib/mrcnn
copying mrcnn/parallel_model.py -> build/lib/mrcnn
copying mrcnn/visualize.py -> build/lib/mrcnn
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/config.p

In [ ]:
!pip show mask-rcnn
train_df = pd.read_csv("train.csv")
train_df.head()

Name: mask-rcnn
Version: 2.1
Summary: Mask R-CNN for object detection and instance segmentation
Home-page: https://github.com/matterport/Mask_RCNN
Author: Matterport
Author-email: waleed.abdulla@gmail.com
License: MIT
Location: /usr/local/lib/python3.6/dist-packages/mask_rcnn-2.1-py3.6.egg
Requires: 
Required-by: 


,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1


In [ ]:
def df_setup(directories):
  train_df = pd.DataFrame(columns=["file_name", "image_id", "height", "width", "anno_list", "annotations"])
  index_dict = {}
  dataset = []

  for dir in directories:
    for filename in os.listdir(dir):
      if(filename.endswith(".jpg")):
        train_df = train_df.append({"file_name": dir + filename, "image_id":filename[:-4], "height":1024, "width":1024, "anno_list":[], "annotations":[]}, ignore_index=True)

  train_df.head()
  return train_df

def get_dataset(filepath1):
  temp_df = pd.read_csv(filepath1)
  temp_df.head()


  directories = ["/content/gdrive/My Drive/Wheat Detection Kaggle/Data/train/", "/content/gdrive/My Drive/Wheat Detection Kaggle/Data/aug_test/"]
  train_df = df_setup(directories)
  print(train_df.head())

  index_dict = {}

  curr_id = temp_df.at[0,"image_id"]
  #train_index = 0
  anno_list = [] # record the annotation of the bbox 

  #update the bounding box list
  for index, row in temp_df.iterrows():
    if row["image_id"] != curr_id:
      # we are in the new image_id now, and we should update the train_df location with the anno_list
      #print(anno_list)
      train_df.loc[train_df['image_id'] == curr_id, "anno_list"] = [anno_list]

      #the annotation should be nulled since now we're in a new image
      anno_list = []  
      curr_id = row["image_id"] # update the curr_id as well
    
    if row["image_id"] == curr_id:
      anno_list.append(row["bbox"])

  dataset = []  # get a new list for our training data since training data can only be dicts

  print(train_df.head())

  #changing the bbox to a type supported by COCO
  for index, row in train_df.iterrows():
    record = dict()
    objs = []  # this objs gather all the bbox we found in the train_df


    for anno in row["anno_list"]:
      print(anno)
      bbox = [float(x) for x in anno[1:-1].split(", ")]
      bbox[2] = bbox[0] + bbox[2]
      bbox[3] = bbox[1] + bbox[3]
      #px = list(range(bbox[0], bbox[0] + bbox[2] + 1))
      #py = list(range(bbox[1], bbox[1] + bbox[3] + 1))

      #poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
      #poly = [p for x in poly for p in x]

      objs.append(bbox)

    # record each of the variable
    record["file_name"] = row["file_name"]
    record["image_id"] = row["image_id"]
    record["height"] = 1024
    record["width"] = 1024
  
    record["annotations"] = objs

    # append
    dataset.append(record)

    index_dict[row["file_name"]] = index

    # print(file_name)
  
  return index_dict, dataset


In [ ]:
index_dict, dataset_dicts = get_dataset("/content/gdrive/My Drive/Wheat Detection Kaggle/Data/train.csv", "/content/gdrive/My Drive/Wheat Detection Kaggle/Data/aug.csv")

print(len(index_dict.keys()))
print(dataset_dicts[:5])

Streaming output truncated to the last 5000 lines.
[403, 565, 93, 90]
[537, 577, 105, 80]
[628, 393, 104, 156]
[748, 417, 80, 77]
[805, 507, 112, 91]
[0, 876, 39, 63]
[39, 854, 96, 170]
[230, 884, 84, 133]
[258, 1002, 94, 22]
[435, 980, 81, 44]
[135, 736, 87, 92]
[222, 659, 69, 81]
[415, 748, 90, 87]
[744, 742, 67, 86]
[842, 799, 89, 108]
[842, 933, 95, 91]
[555, 898, 122, 126]
[500, 933, 110, 91]
[1014, 1006, 10, 18]
[893, 0, 113, 57]
[748, 65, 85, 67]
[685, 6, 89, 51]
[171, 0, 116, 55]
[144, 828, 128, 138]
[952, 955, 47, 69]
[89.0, 822.0, 116.0, 161.0]
[198.0, 808.0, 127.0, 130.0]
[0.0, 542.0, 68.0, 55.0]
[141.0, 534.0, 57.0, 83.0]
[216.0, 458.0, 97.0, 84.0]
[216.0, 266.0, 91.0, 77.0]
[0.0, 286.0, 63.0, 54.0]
[414.0, 163.0, 78.0, 56.0]
[391.0, 107.0, 94.0, 68.0]
[548.0, 0.0, 87.0, 60.0]
[474.0, 217.0, 120.0, 64.0]
[574.0, 111.0, 97.0, 72.0]
[616.0, 240.0, 108.0, 91.0]
[773.0, 277.0, 83.0, 73.0]
[959.0, 68.0, 65.0, 81.0]
[777.0, 105.0, 68.0, 73.0]
[766.0, 0.0, 51.0, 92.0]
[806.0, 0.0,

In [ ]:
class WheatDataset(Dataset):
  # load the dataset definitions
  def load_dataset(self, dataset_dir, is_train=True):
    # define one class
    self.add_class("dataset", 1, "wheat_head")
    # define data locations
    images_dir = ""
    images_dir2 = ""
    if is_train:
      images_dir = dataset_dir + '/train/'
      images_dir2 = dataset_dir + '/aug_test/'
    else:
      images_dir = dataset_dir + "/test/"
    #annotations_dir = dataset_dir + '/annots/'

    # find all images
    for filename in os.listdir(images_dir):
      # extract image id
      image_id = filename[:-4]
      # skip bad images
      img_path = images_dir + filename
      #ann_path = annotations_dir + image_id + '.xml'
      # add to dataset
      # self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)
      self.add_image('dataset', image_id=image_id, path=img_path, annotation=img_path)

    if images_dir2 != "":
      for filename in os.listdir(images_dir2):
        # extract image id
        image_id = filename[:-4]
        # skip bad images
        img_path = images_dir2 + filename
        #ann_path = annotations_dir + image_id + '.xml'
        # add to dataset
        # self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)
        self.add_image('dataset', image_id=image_id, path=img_path, annotation=img_path)

  def extract_boxes(self, path):
    return dataset_dicts[index_dict[path]]["annotations"], dataset_dicts[index_dict[path]]["width"], dataset_dicts[index_dict[path]]["height"]

  # load the masks for an image
  def load_mask(self, image_id):
    # get details of image
    info = self.image_info[image_id]
    # define box file location
    path = info['annotation']
    # load XML
    boxes, w, h = self.extract_boxes(path)
    # create one array for all masks, each on a different channel
    masks = np.zeros([h, w, len(boxes)], dtype='uint8')
    # create masks
    class_ids = list()
    for i in range(len(boxes)):
      box = boxes[i]
      row_s, row_e = int(box[1]), int(box[3])
      col_s, col_e = int(box[0]), int(box[2])
      masks[row_s:row_e, col_s:col_e, i] = 1
      class_ids.append(self.class_names.index('wheat_head'))
    return masks, np.asarray(class_ids, dtype='int32')
 	
  # load an image reference
  def image_reference(self, image_id):
    info = self.image_info[image_id]
    return info["path"]

  

In [ ]:
# train set
train_set = WheatDataset()
train_set.load_dataset('/content/gdrive/My Drive/Wheat Detection Kaggle/Data', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

Train: 10166


In [ ]:
# test/val set
test_set = WheatDataset()
test_set.load_dataset('/content/gdrive/My Drive/Wheat Detection Kaggle/Data', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

Test: 10


In [ ]:
# define image id
image_id = 1
# load the image
image = train_set.load_image(image_id)
# load the masks and the class ids
mask, class_ids = train_set.load_mask(image_id)
# extract bounding boxes from the masks
bbox = extract_bboxes(mask)
# display image with masks and bounding boxes
display_instances(image, bbox, mask, class_ids, train_set.class_names)

IndexError: ignored

In [ ]:
# define a configuration for the model
class WheatConfig(Config):
	# Give the configuration a recognizable name
	NAME = "wheat_cfg"
	# Number of classes (background + wheat)
	NUM_CLASSES = 1 + 1
	# Number of training steps per epoch
	STEPS_PER_EPOCH = 10
 
# prepare config
config = WheatConfig()

In [ ]:
model = MaskRCNN(mode='training', model_dir='/content/gdrive/My Drive/Wheat Detection Kaggle/', config=config)

In [ ]:
# load weights (mscoco)
model.load_weights('mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
#try:
model.train(train_set, train_set, learning_rate=config.LEARNING_RATE, epochs=1, layers='heads')
#except Exception as ex:
  #print(ex)

In [ ]:
# define the prediction configuration
class PredictionConfig(Config):
	# define the name of the configuration
	NAME = "wheat_cfg"
	# number of classes (background + wheat)
	NUM_CLASSES = 1 + 1
	# simplify GPU config
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

	DETECTION_MIN_CONFIDENCE = 0.0
	DETECTION_NMS_THRESHOLD = 0.0

In [ ]:
# create config
cfg = PredictionConfig()
# define the model
model = MaskRCNN(mode='inference', model_dir='/content/gdrive/My Drive/Wheat Detection Kaggle/', config=cfg)
# download the weights
model.load_weights('/content/gdrive/My Drive/Wheat Detection Kaggle/wheat_cfg20200628T0238/mask_rcnn_wheat_cfg_0001.h5', by_name=True)

In [ ]:
# plot a number of photos with ground truth and predictions
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.model import mold_image
from mrcnn.utils import Dataset

#plot actual vs predicted on training dataset
def plot_actual_vs_predicted(dataset, model, cfg, n_images=1):
	# load image and mask
	for i in range(n_images):
		# load the image and mask
		image = dataset.load_image(i)
		mask, _ = dataset.load_mask(i)
		# convert pixel values (e.g. center)
		scaled_image = mold_image(image, cfg)
		# convert image into one sample
		sample = expand_dims(scaled_image, 0)
		# make prediction
		yhat = model.detect(sample, verbose=0)[0]
		# define subplot
		pyplot.subplot(n_images, 2, i*2+1)
		# plot raw pixel data
		pyplot.imshow(image)
		pyplot.title('Actual')
		# plot masks
		for j in range(mask.shape[2]):
			pyplot.imshow(mask[:, :, j], cmap='gray', alpha=0.5)
		# get the context for drawing boxes
		pyplot.subplot(n_images, 2, i*2+2)
		# plot raw pixel data
		pyplot.imshow(image)
		pyplot.title('Predicted')
		ax = pyplot.gca()
		# plot each box
		for box in yhat['rois']:
			# get coordinates
			y1, x1, y2, x2 = box
			# calculate width and height of the box
			width, height = x2 - x1, y2 - y1
			# create the shape
			rect = Rectangle((x1, y1), width, height, fill=False, color='red')
			# draw the box
			ax.add_patch(rect)
	# show the figure
	pyplot.show()
 
#Predict bboxes for images
import cv2
def predict_im(dir):
  image = cv2.cvtColor(cv2.imread(dir), cv2.COLOR_BGR2RGB)
  # convert pixel values (e.g. center)
  scaled_image = mold_image(image, cfg)
  # convert image into one sample
  sample = expand_dims(scaled_image, 0)
  # make prediction
  yhat = model.detect(sample, verbose=0)[0]
  # plot raw pixel data
  pyplot.imshow(image)
  pyplot.title('Image')
  ax = pyplot.gca()
  # plot each box
  for box in yhat['rois']:
    # get coordinates
    y1, x1, y2, x2 = box
    # calculate width and height of the box
    width, height = x2 - x1, y2 - y1
    # create the shape
    rect = Rectangle((x1, y1), width, height, fill=False, color='red')
    # draw the box
    ax.add_patch(rect)
  pyplot.show()

In [ ]:
# plot predictions for train dataset
plot_actual_vs_predicted(train_set, model, cfg)
# plot predictions for test dataset
#plot_actual_vs_predicted(test_set, model, cfg)

predict_im("/content/gdrive/My Drive/Wheat Detection Kaggle/Data/test/2fd875eaa.jpg")

In [ ]:
from mrcnn.utils import compute_ap, compute_ap_range
from numpy import expand_dims
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.model import mold_image
from mrcnn.utils import Dataset

# calculate the mAP for a model on a given dataset
def evaluate_model(dataset, model, cfg):
	APs = list()
	for image_id in dataset.image_ids:
		# load image, bounding boxes and masks for the image id
		image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)
		# convert pixel values (e.g. center)
		scaled_image = mold_image(image, cfg)
		# convert image into one sample
		sample = expand_dims(scaled_image, 0)
		# make prediction
		yhat = model.detect(sample, verbose=0)
		# extract results for first sample
		r = yhat[0]
		# calculate statistics, including AP
		AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
		# store
		APs.append(AP)
	# calculate the mean AP across all images
	mAP = mean(APs)
	return mAP

In [ ]:
mAP = evaluate_model(train_set, model, cfg)
print("The Model's Average Precision (AP) \nmAP: ", AP,n " AP [0.5 to .95]: ", rAP)

In [ ]:
from mrcnn.utils import compute_ap, compute_ap_range
from numpy import expand_dims
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.model import mold_image
from mrcnn.utils import Dataset

import time

gt_boxes = []
gt_class_ids = []
gt_masks = []

pred_boxes = []
pred_class_ids = []
pred_scores = []
pred_masks = []


i = 0
length = len(dataset_dicts)
start_time = time.time()

imgs = []
for record in dataset_dicts:
  #get the predictions for the images
  imgs.append( train_set.load_image(index_dict[record["file_name"]]) )

idx = 0
while idx < len(imgs):
  endx = min(idx + cfg.BATCH_SIZE, len(imgs))
  pred = model.detect(imgs[idx:endx], verbose=0)

pred = 

pred_boxes = pred["rois"]
pred_class_ids = pred["class_ids"]
pred_scores = pred["scores"]
pred_masks = pred["masks"]

#   #get the bboxes for an image
#   gt_boxes.append(record["annotations"])

#   #get the masks and the classes for an image
#   masks, classIDs = train_set.load_mask(index_dict[record["file_name"]])
#   gt_masks.append(masks)
#   gt_class_ids.append(classIDs)

#   i = i+1

#   if i%100 == 0:
#     elapsed_time = time.time() - start_time
#     end_time = elapsed_time/i * length
#     print(i, '/', length, " images processed. Time Elapsed: ", elapsed_time, " Estimated Time: ", end_time)
#   if i == length:
#     print("Processing completed!\n")

load_image_gt(train_set, cfg, )

AP50 = compute_ap(gt_boxes, gt_class_ids, gt_masks,
               pred_boxes, pred_class_ids, pred_scores, pred_masks,
               iou_threshold=0.5)

#Compute AP over a range or IoU thresholds. Default range is 0.5-0.95."""
#Default is 0.5 to 0.95 with increments of 0.05
rAP = compute_ap_range(gt_boxes, gt_class_ids, gt_masks,
                     pred_boxes, pred_class_ids, pred_scores, pred_masks,
                     iou_thresholds=None, verbose=1)

print("The Model's Average Precision (AP) \nAP50: ", AP50, " AP [0.5 to .95]: ", rAP)
